In [1]:
from __future__ import division

In [41]:
def SL_markers_from_sexed_fam(vcf_path, sex_info_file, catalog_path, offspring_presence_threshold, mendelian_threshold, het_sex_thresh, hom_sex_thresh):
    
    """
    Usage: MST_input_maker  <vcf_path>  <sex_info_file>  <offspring_presence_threshold>  <mendelian_threshold>
    
    ### Informative locus filters:
    -Loci are used only if the number of offspring present is equal to or above the percentage specified by the user
    -Loci are used only if they are heterozygous in one sex and homozygous in another.
    
    ###Null allele filters:

    Null allele in Parents:
    -If an offspring is found to have an allele that isn't present in the parent, this locus is discarded

    Null allele in offspring:
    -If only 1 offspring is homozygous for the parental minor allele then that sample is assumed to possess a null allele and the data for that individual is coded as missing. **Note, the sample presence/absence filter works after this step, so some loci may be pushed over the "missing data" limit by this null allele filter. 
    -If two or more samples are found to be homozygous for the minor parental allele, it is more likely that this is due to the homozygous parent having a null allele. In this case, the locus is again discarded.
    -Also, loci with non-mendelian properties indicative of allele dropout or similar are also filtered
    
    """
    #from __future__ import division
    import sys
    import gzip
    import vcf
    from collections import Counter
    import pprint as pp
    from pydoc import help
    

    
    ## Alter the header in the vcf file to be compatible with PyVCF ==========================
    
    alteredvcfpath = "%s%s" % (vcf_path, ".altered")

    oldvcf = open(vcf_path, 'r').readlines()
    alteredvcf = open(alteredvcfpath, 'w')    
    
    for line in oldvcf:
        if "Allele Depth" not in line:
            alteredvcf.write(line)
        elif "Allele Depth" in line:
            line = '##FORMAT=<ID=AD,Number=.,Type=Integer,Description="Allele Depth">\n'
            alteredvcf.write(line)
    alteredvcf.close()
    
    altered_vcf = open(alteredvcfpath, 'r')
    
    ## ========================================================================================
    
    
    #####
    #####
    #####
        
    
    ## Read in VCF, make counters and lists, open output files ================================
    
    myvcf = vcf.Reader(altered_vcf)
        
    Working_dir = "%s/" % (vcf_path.rpartition("/")[0])
    
    loci_used_for_male_map = 0
    loci_used_for_female_map = 0
    
    Loc_w_null_alleles = 0
    loc_w_excess_hom = 0
    
    offspring = []
    male_offspring = []
    female_offspring = []
    
    #ZW_tags = open("%sPutative_ZW_linked_tags.txt" % (MST_path), 'w')
    #XY_tags = open("%sPutative_XY_linked_tags.txt" % (MST_path), 'w')
    log_file = open("%sSL_markers_from_sexed_fam.log" % (Working_dir), 'w')
    
    ZW_tags = []
    XY_tags = []
    
    ## =========================================================================================================

       
    ## Get info for Parents and offspring ======================================================================
    
    sex_file = open(sex_info_file, 'r').readlines()
        
    for line in sex_file: ## Note, only samples in the sex_info file are used
        
        sample_name = line.split()[0]
        sex = line.split()[1]                    
        
        if sex == "F" or sex == "f":
            Mother = sample_name
            log_file.write("Mother = %s" % (sample_name))
        
        elif sex == "M" or sex == "m":
            Father = sample_name
            log_file.write("\nFather = %s\n" % (sample_name))        
    
        elif sex == "O" or sex == "o" or sex == "MO" or sex == "FO" or sex == "mo" or sex == "fo":
            offspring.append(sample_name)
            if sex == "MO" or  sex == "mo":
                male_offspring.append(sample_name)
            elif sex == "FO" or  sex == "fo":
                female_offspring.append(sample_name)
            
    offspring = sorted(offspring) ## sort the list of offspring - used to retain correct order throughout script
    male_offspring = sorted(male_offspring)
    female_offspring = sorted(female_offspring)
    
    
    
    ## ===========================================================================================================

    
    ## Looping over loci to get info =============================================================================
    ## First check loci for signs of wrong genotype calls, assign them to a list of "Good loci".
    ## And then check these good loci for XY or ZW signal    
    
    Good_loci = []
    
    Male_offs_GTs = {}
    Female_offs_GTs = {}
    N_uncalled_male_offs = 0
    N_uncalled_female_offs = 0
    
    
    N_ZW_testers = 0
    N_ZW_testers_with_enough_het_females = 0
    N_ZW_testers_with_enough_het_females_and_hom_males = 0
            
            
    N_XY_testers = 0
    N_XY_testers_with_enough_het_males = 0
    N_XY_testers_with_enough_het_males_and_hom_females = 0

    for record in myvcf:
        
        ## Make all variable defaults, lists and dicts.
        
        Mat_ref = None
        Mat_alt = None
        Pat_ref = None
        Pat_alt = None
        mat_het = None
        pat_het = None
        Mat_genotype = None
        Pat_genotype = None
        Mat_genotype_bases = None
        Pat_genotype_bases = None
        Non_par_allele = False
        N_offspring = 0
        offspr_w_allele_dropout = 0
        N_offsp_hom_for_minor_par_allele = 0
        
        Sample_headers = []
        sample_names = []
        Parental_alleles = []
        offspring_gt_types = []
        
        offspring_gt_types_dict = {}
        offspring_gt_bases_dict = {}
        allele_dropout_scores = {}
        
        ## lists for constructing the lines of the MST output file
        malemap_offspring_haploid_genotypes = []
        malemap_offspring_haploid_genotypes_compli = []
        femalemap_offspring_haploid_genotypes = []
        femalemap_offspring_haploid_genotypes_compli = []
    
    
        Loc_Id = "%s_%s" % (record.ID, record.POS)
        
        log_file.write("\n\nLocus: %s -------------\n" % (Loc_Id))
        
        for sample in record.samples:
            
            name = sample.sample
            genotype = sample['GT']
            allele_dropout_scores[record] = {}
            
            sample_names.append(name)
            
            
            ## Get Mother and Father information
            
            if name == Mother:
                
                if not sample.called == False:
                    Mat_ref = sample.gt_bases.split("/")[0]
                    Mat_alt = sample.gt_bases.split("/")[1]
                    Mat_genotype = genotype
                    Mat_genotype_bases = sample.gt_bases
                    
                    Parental_alleles.append(Mat_ref)
                    Parental_alleles.append(Mat_alt)
                    
                    if sample.gt_type == 0 or sample.gt_type == 2:
                        mat_het = False
                        log_file.write("\nMother is homozygous (%s)" % (sample.gt_bases))
                    elif sample.gt_type == 1:
                        mat_het = True   
                        log_file.write("\nMother is heterozygous (%s)" % (sample.gt_bases))
                elif sample.called == False:
                    log_file.write("\nLocus not called in mother")
                    
                
                    
            elif name == Father:
                
                if not sample.called == False:
                    Pat_ref = sample.gt_bases.split("/")[0]
                    Pat_alt = sample.gt_bases.split("/")[1]
                    Pat_genotype = genotype
                    Pat_genotype_bases = sample.gt_bases
                    
                    Parental_alleles.append(Pat_ref)
                    Parental_alleles.append(Pat_alt)
                    
                    if sample.gt_type == 0 or sample.gt_type == 2:
                        pat_het = False
                        log_file.write("\nFather is homozygous (%s)" % (sample.gt_bases))
                    elif sample.gt_type == 1:
                        pat_het = True
                        log_file.write("\nFather is heterozygous (%s)" % (sample.gt_bases))
                elif sample.called == False:
                    log_file.write("\nLocus not called in father")
            
        
        ## Get parental allele information and filter for unacceptable loci
        
        Par_allele_counts = Counter(Parental_alleles)
        
        
        if None in Par_allele_counts or Mat_genotype == None or Pat_genotype == None: ## If there is a missing call in the parents, discard locus
            log_file.write("\nMissing data in parents, locus discarded")
                        
        elif len(Par_allele_counts) > 2: ## If there are more than two alleles in the parents, discard locus
            log_file.write("\nparents contain more than 2 alleles, locus not used")
            
        elif pat_het == True and mat_het == True: ## If both parents are heterozygous, discard locus
            log_file.write("\nBoth parents heterozygous, locus not used")
            
        elif pat_het == False and mat_het == False: ## If both parents are homozygous, discard locus
            log_file.write("\nBoth parents homozygous, locus not used")
            

            
        elif len(Par_allele_counts) == 2 and not 2 in Par_allele_counts.values(): ## if both parents are called and both do not have the same genotype
            
            for allele in Par_allele_counts:
                if Par_allele_counts[allele] == 3:
                    Par_major_allele = allele
                elif Par_allele_counts[allele] == 1:
                    Par_minor_allele = allele
                    
        
            ## Get offspring information -----------------------------------------------------------------------
                 
            
            ## First - look for signs of allele dropout - using all offspring for this
            
            for sample in record.samples:            
                name = sample.sample
                genotype = sample['GT']            
                
                if name in offspring:  ## Only samples included in the pop_map file are used
                    N_offspring += 1


                    ### Filter sample genotypes for signs of allele droupout 

                    if genotype == None: ## If sample not called, record the lack of genotype
                        
                        offspring_gt_types.append(sample.gt_type) # record hom or het
                        offspring_gt_bases_dict[name] = (sample.gt_type, sample.gt_bases)
                        
                        
                    elif not genotype == None: ## If sample has been called

                        off_ref = sample.gt_bases.split("/")[0] ## get the samples ref allele
                        off_alt = sample.gt_bases.split("/")[1] ## get the samples alt allele
                        
                        if sample.gt_bases == "%s/%s" % (Par_minor_allele,Par_minor_allele): ## If sample homozygous for the parental minor allele, remove sample info, record the event for tallying later.
                            
                            log_file.write("\n%s is homozygous for minor parental allele (%s), likely allele dropout in offspring (if rare) or parent (if common)" % (name, sample.gt_bases))
                            offspr_w_allele_dropout += 1

                            offspring_gt_types.append(None) # Count as missing data
                            offspring_gt_bases_dict[name] = (None, None)
                            N_offsp_hom_for_minor_par_allele += 1

                        elif off_ref not in Parental_alleles or off_alt not in Parental_alleles: ## If sample has an allele that is not in parents, remove locus
                            
                            offspring_gt_types.append(sample.gt_type) # record hom or het
                            offspring_gt_bases_dict[name] = (sample.gt_type, sample.gt_bases)

                            log_file.write("\n%s contains non-parental allele in genotype (%s), likely allele dropout in a Parent" % (name, sample.gt_bases))
                            Non_par_allele = True


                        if off_ref in Parental_alleles and off_alt in Parental_alleles: ## And if the sample's alleles are both in the Parental genotypes then the locus is good
                            
                            offspring_gt_types.append(sample.gt_type) # record hom or het
                            offspring_gt_bases_dict[name] = (sample.gt_type, sample.gt_bases)

                            ## Record male and female genotypes for later use if the locus is ok.

                            if name in male_offspring:
                                if Loc_Id not in Male_offs_GTs:
                                    Male_offs_GTs[Loc_Id] = {}
                                Male_offs_GTs[Loc_Id][name] = (sample.gt_type, sample.gt_bases)

                                if sample.gt_type == None:
                                    N_uncalled_male_offs += 1

                            elif name in female_offspring:
                                if Loc_Id not in Female_offs_GTs:
                                    Female_offs_GTs[Loc_Id] = {}
                                Female_offs_GTs[Loc_Id][name] = (sample.gt_type, sample.gt_bases)

                                if sample.gt_type == None:
                                    N_uncalled_female_offs += 1

            
            counted = Counter(offspring_gt_types) ## Count the numbers of each genotype at this locus for looking at offspring presence threshold

            

            ## Caluculate the percentage of offspring missing at the locus

            if None in counted:
                perc_offspring_missing = counted[None]/N_offspring
            else:
                perc_offspring_missing = 0

            ## calculate the percentages of homozygotes/heterozygotes for mendelian frequency filters below

            perc_hom_0 = counted[0]/(counted[0]+counted[2]+counted[1])
            perc_hom_2 = counted[2]/(counted[0]+counted[2]+counted[1])
            perc_het = counted[1]/(counted[0]+counted[2]+counted[1])
            perc_off_w_all_dropout = offspr_w_allele_dropout/N_offspring                  

            ## Filter loci with non-mendelian genotype proportions and write data to repsective map files

            if perc_offspring_missing > 1-offspring_presence_threshold:
                log_file.write("\nToo many missing genotypes, locus not used")
                #print "Too many missing genotypes, locus not used"

            elif Non_par_allele == True:
                log_file.write("\nNon parental allele found, locus not used")
                #print "Non parental allele found, locus not used"

            elif N_offsp_hom_for_minor_par_allele > 1: ## if there is more than one offspring homozygous for the minor parental allele, then it is likely to be allele dropout in the parent, locus is discarded
                log_file.write("\nMore than one offspring homozygous for minor allele, locus not used")
                #print "More than one offspring homozygous for minor allele, locus not used"
            elif perc_hom_0 > mendelian_threshold or perc_hom_2 > mendelian_threshold:
                log_file.write("\nHomozygosity excess (>%s)" % (mendelian_threshold))
                log_file.write("\nLocus discarded, Perc_hom_0= %s, Perc_hom_2= %s" % (perc_hom_0, perc_hom_2))
                loc_w_excess_hom += 1
                #print "Homozygosity excess"
            else:
                #print "Locus passed null allele filters"
                Good_loci.append(Loc_Id)


            ## Now look through offspring again, to test for XY, look only at loci where Father is heterozygous and find those that 
            ## are heterozygous in all (or more than a threshold number) of males

            ##============================================================================================================================= 
            ## Now look specifically at loci that are het in only one parent. =============================================================
            ## ============================================================================================================================

            ## Get total number of CALLED females

            N_called_fem_offspring = len(female_offspring) - N_uncalled_female_offs
            N_called_male_offspring = len(male_offspring) - N_uncalled_male_offs
            
            

            

            if Loc_Id in Good_loci:            

                if pat_het == False and mat_het == True: ## If mother is het and father is hom, use to test for ZW
                    log_file.write("\nMother is heterozygous, Father is homozygous, locus used to test for ZW")
                    
                    N_ZW_testers += 1
                    
                    ## Now look through the male and female offspring - look for ZW patterns

                    N_het_females = 0
                    N_hom_males = 0

                    ## Find out the percentage of females called that were heterozygous
                    
                    for female in Female_offs_GTs[Loc_Id]:
                        if Female_offs_GTs[Loc_Id][female][0] == 1 and Mat_alt in Female_offs_GTs[Loc_Id][female][1].split("/"): ## If female is heterozygous count it. 
                            N_het_females += 1
                    
                    ## And then find out the percentage of males called that were homozygous
                    
                    for male in Male_offs_GTs[Loc_Id]:
                        if Male_offs_GTs[Loc_Id][male][0] in [0,2]: ## If male is homozygous count it. 
                            N_hom_males += 1
                        
                    

                    if N_het_females > 1:
                        perc_het_females = N_het_females/N_called_fem_offspring
                        log_file.write("\nN called female offspring = %s" % N_called_fem_offspring)
                        log_file.write("\nN heterozygous females = %s" % N_het_females)
                        log_file.write("\nPercent heterozygous female offspring = %s" % perc_het_females)
                        
                        N_ZW_testers_with_enough_het_females += 1
                        
                        if perc_het_females >= het_sex_thresh:
                            log_file.write("\nEnough heterozygous females!")

                            if N_hom_males > 1:
                                perc_hom_males = N_hom_males/N_called_male_offspring
                                log_file.write("\nPercent homozygous males = %s" % perc_hom_males)
                                
                                if perc_hom_males >= hom_sex_thresh:
                                    ZW_tags.append(Loc_Id)
                                    log_file.write("\nEnough homozygous males!")
                                    log_file.write("\n##Locus supports ZW system!")
                                    N_ZW_testers_with_enough_het_females_and_hom_males += 1
                                else:
                                    log_file.write("\nNot enough homozygous males")
                            else:
                                log_file.write("\nNo homozygous male offspring")
                        else:
                            log_file.write("\nNot enough heterozygous female offspring")
                    else:
                        log_file.write("\nNo heterozygous female offspring")

                elif pat_het == True and mat_het == False: ## If father is het and mother is hom, use to test for XY
                    log_file.write("\nFather is heterozygous, Mother is homozygous, locus used to test for XY")
                    N_XY_testers += 1
                    
                    
                    N_het_males = 0
                    N_hom_females = 0
                    
                    ## Find out the percentage of males called that were heterozygous
                    
                    for male in Male_offs_GTs[Loc_Id]:
                        if Male_offs_GTs[Loc_Id][male][0] == 1 and Pat_alt in Male_offs_GTs[Loc_Id][male][1].split("/"): ## If male is heterozygous count it. 
                            N_het_males += 1
                    
                    ## Find out the percentage of females called that were homozygous
                    
                    for female in Female_offs_GTs[Loc_Id]:
                        if Female_offs_GTs[Loc_Id][female][0] in [0,2]: ## If female is homozygous count it. 
                            N_hom_females += 1
                            
                    

                    if N_het_males > 1:
                        perc_het_males = N_het_males/N_called_male_offspring
                        
                        log_file.write("\nN called male offspring = %s" % N_called_male_offspring)
                        log_file.write("\nN heterozygous male offspring = %s" % N_het_males)
                        log_file.write("\nPercent heterozygous males = %s" % perc_het_males)
                        
                        if perc_het_males >= het_sex_thresh:
                            log_file.write("\nEnough heterozygous males!")
                            N_XY_testers_with_enough_het_males += 1
                            
                            if N_hom_females > 1:
                                perc_hom_females = N_hom_females/N_called_fem_offspring
                                log_file.write("\nPercent homozygous females = %s" % perc_hom_females)
                                
                                if perc_hom_females >= hom_sex_thresh:
                                    XY_tags.append(Loc_Id)
                                    log_file.write("\nEnough homozygous females!")
                                    log_file.write("\n##Locus supports XY system!")
                                    N_XY_testers_with_enough_het_males_and_hom_females += 1
                                else:
                                    log_file.write("\nNot enough homozygous females")

                            else:
                                log_file.write("\nNo homozygous female offspring")        
                        else:
                            log_file.write("\nNot enough heterozygous male offspring")           
                    else:
                        log_file.write("\nNo heterozygous male offspring")


    print "Number of good loci = %s" % len(Good_loci)

    print "\nN loci suitable for XY testing:", N_XY_testers
    print "N XY test loci with enough heterozygous males:", N_XY_testers_with_enough_het_males
    print "N loci that fit the specified XY criteria:", N_XY_testers_with_enough_het_males_and_hom_females
    
    print "\nN loci suitable for ZW testing:", N_ZW_testers
    print "N ZW test loci with enough heterozygous females:", N_ZW_testers_with_enough_het_females
    print "N loci that fit the specified ZW criteria:", N_ZW_testers_with_enough_het_females_and_hom_males
    

    
    ## Finally, make fasta files and vcf files of the sex linked markers
    
    XYset = set(XY_tags)
    ZWset = set(ZW_tags)
    
    if catalog_path.endswith("gz"):
        catalog = gzip.open(catalog_path, 'r').readlines()
    else:
        catalog = open(catalog_path, 'r').readlines()

        
    ## Outdir same as vcf dir:

    outdir = vcf_path.rpartition("/")[0]

    if len(XYset) > 0:
        Putative_XYlinked_makers_file = open("%s/%s" %(outdir, "Putative_XYlinked_makers.fa"), 'w')

        for locus in XYset:
            for tag in catalog:
                if locus.split("_")[0] == tag.split()[2]:
                    Putative_XYlinked_makers_file.write(">X_linkedLocusID_%s\n" % locus)
                    Putative_XYlinked_makers_file.write("%s\n" % (tag.split()[8]))
        Putative_XYlinked_makers_file.close()

    if len(ZWset) > 0:
        Putative_ZWlinked_makers_file = open("%s/%s" %(outdir, "Putative_ZWlinked_makers.fa"), 'w')

        for locus in ZWset:
            for tag in catalog:
                if locus.split("_")[0] == tag.split()[2]:
                    Putative_ZWlinked_makers_file.write(">Z_linked|LocusID_%s\n" % locus)
                    Putative_ZWlinked_makers_file.write("%s\n" % (tag.split()[8]))
        Putative_ZWlinked_makers_file.close()

    return XY_tags, ZW_tags
        

In [42]:
## Analyses parameters

vcf_path = "/home/djeffrie/Data/RADseq/STOECK/Bviridis/Final_SNP_data/batch_1_strict.vcf.altered"
popmap_path = "/home/djeffrie/Data/RADseq/STOECK/Bviridis/Final_SNP_data/popmap.txt"
catalog_file = "/home/djeffrie/Data/RADseq/STOECK/Bviridis/Final_SNP_data/batch_1.catalog.tags.tsv.gz"
offspring_presence_threshold = 0.9   # 90% of offspring must have data for a given locus
medelian_cut_off = 0.75   # 75% mendelian segregation frequency threshold
min_thresh_for_heterozygous_heterogametic_sex = 0.75  # 75% of the heterogametic sex must be heterozygous
min_thresh_for_homozygous_homogametic_sex = 0.9  # 90% of the homogametic sex must be homozygous


XYs, ZWs = SL_markers_from_sexed_fam(vcf_path,
                                     popmap_path, 
                                     catalog_file, 
                                     offspring_presence_threshold, 
                                     medelian_cut_off, 
                                     min_thresh_for_heterozygous_heterogametic_sex, 
                                     min_thresh_for_homozygous_homogametic_sex)



Number of good loci = 2250

N loci suitable for XY testing: 1349
N XY test loci with enough heterozygous males: 219
N loci that fit the specified XY criteria: 137

N loci suitable for ZW testing: 901
N ZW test loci with enough heterozygous females: 896
N loci that fit the specified ZW criteria: 0


Something wrong with the rpy2 thing, so the plotting isn't working at the moment. Fix next week. In the mean time . . . it looks from the PCA like there is some misassignment. Also one of the females stands out a bit, which might mean it was from a different family. Re-run the analyses without these two samples. . . . (Still have 8 females and 7 males in the offspring)

In [17]:
## Analyses parameters

vcf_path = "/home/djeffrie/Data/RADseq/STOECK/Bviridis/Final_SNP_data/batch_1_strict.vcf.altered"
popmap_path = "/home/djeffrie/Data/RADseq/STOECK/Bviridis/Final_SNP_data/pop_map_edited.txt"
catalog_file = ""
offspring_presence_threshold = 0.9   # 90% of offspring must have data for a given locus
medelian_cut_off = 0.75   # 75% mendelian segregation frequency threshold
min_thresh_for_heterozygous_heterogametic_sex = 0.75  # 75% of the heterogametic sex must be heterozygous
min_thresh_for_homozygous_homogametic_sex = 0.9  # 90% of the homogametic sex must be homozygous


XYs, ZWs = MST_input_maker(vcf_path,
                           popmap_path, 
                           offspring_presence_threshold, 
                           medelian_cut_off, 
                           min_thresh_for_heterozygous_heterogametic_sex, 
                           min_thresh_for_homozygous_homogametic_sex)



Number of good loci = 2250

N loci suitable for XY testing: 1349
N XY test loci with enough heterozygous males: 276
N loci that fit the specified XY criteria: 144

N loci suitable for ZW testing: 901
N ZW test loci with enough heterozygous females: 886
N loci that fit the specified ZW criteria: 2
